# INSY669 - Text Analytics
## Course Project - Amazon Glassdoor Reviews

### Import libraries

In [2]:
import numpy as np
import pandas as pd
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

import gensim.corpora as corpora
import gensim, logging, warnings
import matplotlib.colors as mcolors

import pprint
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

### Import data

In [3]:
amzn = pd.read_csv('C:\\Users\\m.maraqa\\Desktop\\MMA courses\\Winter 2023\\Text Analytics\\Project\\Amazon_comments.csv')

In [5]:
# no processing on ID
ids = amzn['id']
amzn = amzn.drop('id', axis = 1)

In [7]:
amzn

,Title_Review,Stars,Company_name,Recommend,CEO_approval,Business_outlook,Pros,Cons,Employee_seniority,Location,Date
0,good impression in the first months,5.0,Amazon,positive,neutral,neutral,Documentation on Amazon is a super important p...,You need to understand your job and what you n...,"Current Employee, less than 1 year","Toronto, ON","Feb. 2, 2023"
1,intern,5.0,Amazon,neutral,neutral,neutral,4 days shifts is nice,long hour shift can make you feel tired,"Former Employee, less than 1 year","Toronto, ON","Jan. 31, 2023"
2,good,5.0,Amazon,positive,positive,positive,"great work balance, great environment, locatio...",workload can be heavy sometimes,"Former Employee, more than 1 year","Amazon, SK","Jan. 31, 2023"
3,job review,5.0,Amazon,positive,positive,positive,"good benefit, flexible time shift. take care o...","better organization of work, better car parkin...",Former Employee,NaN,"Jan. 31, 2023"
4,growth opportunity,4.0,Amazon,positive,negative,negative,"Fast paced, Start-Up Culture, Benefits","Compensation, Growth Prospects, Development Op...","Current Employee, more than 3 years","Vancouver, BC","Jan. 31, 2023"
...,...,...,...,...,...,...,...,...,...,...,...
9995,great pay and onboarding,5.0,Amazon,positive,positive,positive,Amazon has wonderful search sites where you ca...,The interview process is very long but worth i...,"Current Employee, less than 1 year",NaN,"Dec. 1, 2022"
9996,great comp,5.0,Amazon,neutral,neutral,neutral,great company easy to find an area you like,can get unlucky with team.,Current Employee,NaN,"Dec. 1, 2022"
9997,"so far,so good!",5.0,Amazon,positive,positive,positive,"great teamwork, great working environment, per...",a little far from home,"Current Employee, less than 1 year","Querétaro, Querétaro","Dec. 1, 2022"
9998,used to be a great company,2.0,Amazon,negative,negative,negative,You will become an excellent problem solver us...,"Cut-throat management and toxic culture, unnec...","Former Employee, more than 1 year","Diego, CA","Dec. 15, 2022"


### Lowercase the Strings

In [ ]:
# extract string columns
txt = amzn.select_dtypes(include = object)

In [32]:
# apply lowercase to all string columns
for i in txt.columns:
    low = []
    amzn[i] = amzn[i].astype(str)
    for j in range(len(amzn)):
        low.append(amzn[i][j].lower())
    amzn[i] = low
    
amzn

,Title_Review,Stars,Company_name,Recommend,CEO_approval,Business_outlook,Pros,Cons,Employee_seniority,Location,Date
0,good impression in the first months,5.0,amazon,positive,neutral,neutral,documentation on amazon is a super important p...,you need to understand your job and what you n...,"current employee, less than 1 year","toronto, on","feb. 2, 2023"
1,intern,5.0,amazon,neutral,neutral,neutral,4 days shifts is nice,long hour shift can make you feel tired,"former employee, less than 1 year","toronto, on","jan. 31, 2023"
2,good,5.0,amazon,positive,positive,positive,"great work balance, great environment, locatio...",workload can be heavy sometimes,"former employee, more than 1 year","amazon, sk","jan. 31, 2023"
3,job review,5.0,amazon,positive,positive,positive,"good benefit, flexible time shift. take care o...","better organization of work, better car parkin...",former employee,nan,"jan. 31, 2023"
4,growth opportunity,4.0,amazon,positive,negative,negative,"fast paced, start-up culture, benefits","compensation, growth prospects, development op...","current employee, more than 3 years","vancouver, bc","jan. 31, 2023"
...,...,...,...,...,...,...,...,...,...,...,...
9995,great pay and onboarding,5.0,amazon,positive,positive,positive,amazon has wonderful search sites where you ca...,the interview process is very long but worth i...,"current employee, less than 1 year",nan,"dec. 1, 2022"
9996,great comp,5.0,amazon,neutral,neutral,neutral,great company easy to find an area you like,can get unlucky with team.,current employee,nan,"dec. 1, 2022"
9997,"so far,so good!",5.0,amazon,positive,positive,positive,"great teamwork, great working environment, per...",a little far from home,"current employee, less than 1 year","querétaro, querétaro","dec. 1, 2022"
9998,used to be a great company,2.0,amazon,negative,negative,negative,you will become an excellent problem solver us...,"cut-throat management and toxic culture, unnec...","former employee, more than 1 year","diego, ca","dec. 15, 2022"


### Remove Stopwords

In [34]:
stop_words = set(stopwords.words('english'))
punctuation_marks = set(string.punctuation)
for i in ['“','”','—','’','`','``','\'','\""']:
    punctuation_marks.add(i)

# define a function to remove stop words and punctuation marks
def remove_stop_words_and_punctuation(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words and word not in punctuation_marks]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# apply the function to the 'text' column of the dataframe
for i in txt.columns:
    amzn[i] = amzn[i].apply(remove_stop_words_and_punctuation)
    
amzn

,Title_Review,Stars,Company_name,Recommend,CEO_approval,Business_outlook,Pros,Cons,Employee_seniority,Location,Date
0,good impression first months,5.0,amazon,positive,neutral,neutral,documentation amazon super important point pro...,need understand job need improve better depend...,current employee less 1 year,toronto,feb. 2 2023
1,intern,5.0,amazon,neutral,neutral,neutral,4 days shifts nice,long hour shift make feel tired,former employee less 1 year,toronto,jan. 31 2023
2,good,5.0,amazon,positive,positive,positive,great work balance great environment location far,workload heavy sometimes,former employee 1 year,amazon sk,jan. 31 2023
3,job review,5.0,amazon,positive,positive,positive,good benefit flexible time shift take care emp...,better organization work better car parking ar...,former employee,nan,jan. 31 2023
4,growth opportunity,4.0,amazon,positive,negative,negative,fast paced start-up culture benefits,compensation growth prospects development oppo...,current employee 3 years,vancouver bc,jan. 31 2023
...,...,...,...,...,...,...,...,...,...,...,...
9995,great pay onboarding,5.0,amazon,positive,positive,positive,amazon wonderful search sites find anything lo...,interview process long worth end,current employee less 1 year,nan,dec. 1 2022
9996,great comp,5.0,amazon,neutral,neutral,neutral,great company easy find area like,get unlucky team,current employee,nan,dec. 1 2022
9997,far good,5.0,amazon,positive,positive,positive,great teamwork great working environment perso...,little far home,current employee less 1 year,querétaro querétaro,dec. 1 2022
9998,used great company,2.0,amazon,negative,negative,negative,become excellent problem solver using data con...,cut-throat management toxic culture unnecessar...,former employee 1 year,diego ca,dec. 15 2022


### Tokenizing String

In [35]:
for i in txt.columns:
    amzn[i] = amzn[i].apply(nltk.word_tokenize)
    
amzn

,Title_Review,Stars,Company_name,Recommend,CEO_approval,Business_outlook,Pros,Cons,Employee_seniority,Location,Date
0,"[good, impression, first, months]",5.0,[amazon],[positive],[neutral],[neutral],"[documentation, amazon, super, important, poin...","[need, understand, job, need, improve, better,...","[current, employee, less, 1, year]",[toronto],"[feb., 2, 2023]"
1,[intern],5.0,[amazon],[neutral],[neutral],[neutral],"[4, days, shifts, nice]","[long, hour, shift, make, feel, tired]","[former, employee, less, 1, year]",[toronto],"[jan., 31, 2023]"
2,[good],5.0,[amazon],[positive],[positive],[positive],"[great, work, balance, great, environment, loc...","[workload, heavy, sometimes]","[former, employee, 1, year]","[amazon, sk]","[jan., 31, 2023]"
3,"[job, review]",5.0,[amazon],[positive],[positive],[positive],"[good, benefit, flexible, time, shift, take, c...","[better, organization, work, better, car, park...","[former, employee]",[nan],"[jan., 31, 2023]"
4,"[growth, opportunity]",4.0,[amazon],[positive],[negative],[negative],"[fast, paced, start-up, culture, benefits]","[compensation, growth, prospects, development,...","[current, employee, 3, years]","[vancouver, bc]","[jan., 31, 2023]"
...,...,...,...,...,...,...,...,...,...,...,...
9995,"[great, pay, onboarding]",5.0,[amazon],[positive],[positive],[positive],"[amazon, wonderful, search, sites, find, anyth...","[interview, process, long, worth, end]","[current, employee, less, 1, year]",[nan],"[dec., 1, 2022]"
9996,"[great, comp]",5.0,[amazon],[neutral],[neutral],[neutral],"[great, company, easy, find, area, like]","[get, unlucky, team]","[current, employee]",[nan],"[dec., 1, 2022]"
9997,"[far, good]",5.0,[amazon],[positive],[positive],[positive],"[great, teamwork, great, working, environment,...","[little, far, home]","[current, employee, less, 1, year]","[querétaro, querétaro]","[dec., 1, 2022]"
9998,"[used, great, company]",2.0,[amazon],[negative],[negative],[negative],"[become, excellent, problem, solver, using, da...","[cut-throat, management, toxic, culture, unnec...","[former, employee, 1, year]","[diego, ca]","[dec., 15, 2022]"


### Part of Speech Tagging

In [36]:
for i in txt.columns:
    amzn[i] = amzn[i].apply(nltk.pos_tag)
    
amzn

,Title_Review,Stars,Company_name,Recommend,CEO_approval,Business_outlook,Pros,Cons,Employee_seniority,Location,Date
0,"[(good, JJ), (impression, NN), (first, RB), (m...",5.0,"[(amazon, NN)]","[(positive, JJ)]","[(neutral, JJ)]","[(neutral, JJ)]","[(documentation, NN), (amazon, VBZ), (super, J...","[(need, NN), (understand, VBP), (job, NN), (ne...","[(current, JJ), (employee, NN), (less, RBR), (...","[(toronto, NN)]","[(feb., RB), (2, CD), (2023, CD)]"
1,"[(intern, NN)]",5.0,"[(amazon, NN)]","[(neutral, JJ)]","[(neutral, JJ)]","[(neutral, JJ)]","[(4, CD), (days, NNS), (shifts, JJ), (nice, JJ)]","[(long, RB), (hour, NN), (shift, NN), (make, V...","[(former, JJ), (employee, NN), (less, RBR), (1...","[(toronto, NN)]","[(jan., NN), (31, CD), (2023, CD)]"
2,"[(good, JJ)]",5.0,"[(amazon, NN)]","[(positive, JJ)]","[(positive, JJ)]","[(positive, JJ)]","[(great, JJ), (work, NN), (balance, NN), (grea...","[(workload, NN), (heavy, NN), (sometimes, RB)]","[(former, JJ), (employee, NN), (1, CD), (year,...","[(amazon, NN), (sk, NN)]","[(jan., NN), (31, CD), (2023, CD)]"
3,"[(job, NN), (review, NN)]",5.0,"[(amazon, NN)]","[(positive, JJ)]","[(positive, JJ)]","[(positive, JJ)]","[(good, JJ), (benefit, NN), (flexible, JJ), (t...","[(better, JJR), (organization, NN), (work, NN)...","[(former, JJ), (employee, NN)]","[(nan, NN)]","[(jan., NN), (31, CD), (2023, CD)]"
4,"[(growth, NN), (opportunity, NN)]",4.0,"[(amazon, NN)]","[(positive, JJ)]","[(negative, JJ)]","[(negative, JJ)]","[(fast, RB), (paced, JJ), (start-up, JJ), (cul...","[(compensation, NN), (growth, NN), (prospects,...","[(current, JJ), (employee, NN), (3, CD), (year...","[(vancouver, NN), (bc, NN)]","[(jan., NN), (31, CD), (2023, CD)]"
...,...,...,...,...,...,...,...,...,...,...,...
9995,"[(great, JJ), (pay, NN), (onboarding, VBG)]",5.0,"[(amazon, NN)]","[(positive, JJ)]","[(positive, JJ)]","[(positive, JJ)]","[(amazon, RB), (wonderful, JJ), (search, NN), ...","[(interview, NN), (process, NN), (long, RB), (...","[(current, JJ), (employee, NN), (less, RBR), (...","[(nan, NN)]","[(dec., NN), (1, CD), (2022, CD)]"
9996,"[(great, JJ), (comp, NN)]",5.0,"[(amazon, NN)]","[(neutral, JJ)]","[(neutral, JJ)]","[(neutral, JJ)]","[(great, JJ), (company, NN), (easy, JJ), (find...","[(get, VB), (unlucky, JJ), (team, NN)]","[(current, JJ), (employee, NN)]","[(nan, NN)]","[(dec., NN), (1, CD), (2022, CD)]"
9997,"[(far, RB), (good, JJ)]",5.0,"[(amazon, NN)]","[(positive, JJ)]","[(positive, JJ)]","[(positive, JJ)]","[(great, JJ), (teamwork, NN), (great, JJ), (wo...","[(little, JJ), (far, RB), (home, NN)]","[(current, JJ), (employee, NN), (less, RBR), (...","[(querétaro, NN), (querétaro, NN)]","[(dec., NN), (1, CD), (2022, CD)]"
9998,"[(used, VBN), (great, JJ), (company, NN)]",2.0,"[(amazon, NN)]","[(negative, JJ)]","[(negative, JJ)]","[(negative, JJ)]","[(become, VB), (excellent, JJ), (problem, NN),...","[(cut-throat, JJ), (management, NN), (toxic, J...","[(former, JJ), (employee, NN), (1, CD), (year,...","[(diego, NN), (ca, MD)]","[(dec., NN), (15, CD), (2022, CD)]"


### Lemmatization

In [41]:
lemmatizer = WordNetLemmatizer()
def wordnet_pos(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    else:
        return wordnet.ADJ_SAT

def lemmatize_text(pos_tags):
    return [lemmatizer.lemmatize(token, wordnet_pos(pos_tag)).lower() for (token, pos_tag) in pos_tags]

for i in txt.columns:
    lem = []
    for j in range(len(amzn)):
        lem.append(lemmatize_text(amzn[i][j]))
    
    amzn[i] = lem
    
amzn

,Title_Review,Stars,Company_name,Recommend,CEO_approval,Business_outlook,Pros,Cons,Employee_seniority,Location,Date
0,"[good, impression, first, month]",5.0,[amazon],[positive],[neutral],[neutral],"[documentation, amazon, super, important, poin...","[need, understand, job, need, improve, good, d...","[current, employee, less, 1, year]",[toronto],"[feb., 2, 2023]"
1,[intern],5.0,[amazon],[neutral],[neutral],[neutral],"[4, day, shifts, nice]","[long, hour, shift, make, feel, tire]","[former, employee, less, 1, year]",[toronto],"[jan., 31, 2023]"
2,[good],5.0,[amazon],[positive],[positive],[positive],"[great, work, balance, great, environment, loc...","[workload, heavy, sometimes]","[former, employee, 1, year]","[amazon, sk]","[jan., 31, 2023]"
3,"[job, review]",5.0,[amazon],[positive],[positive],[positive],"[good, benefit, flexible, time, shift, take, c...","[good, organization, work, well, car, parking,...","[former, employee]",[nan],"[jan., 31, 2023]"
4,"[growth, opportunity]",4.0,[amazon],[positive],[negative],[negative],"[fast, paced, start-up, culture, benefit]","[compensation, growth, prospect, development, ...","[current, employee, 3, year]","[vancouver, bc]","[jan., 31, 2023]"
...,...,...,...,...,...,...,...,...,...,...,...
9995,"[great, pay, onboarding]",5.0,[amazon],[positive],[positive],[positive],"[amazon, wonderful, search, site, find, anythi...","[interview, process, long, worth, end]","[current, employee, less, 1, year]",[nan],"[dec., 1, 2022]"
9996,"[great, comp]",5.0,[amazon],[neutral],[neutral],[neutral],"[great, company, easy, find, area, like]","[get, unlucky, team]","[current, employee]",[nan],"[dec., 1, 2022]"
9997,"[far, good]",5.0,[amazon],[positive],[positive],[positive],"[great, teamwork, great, work, environment, pe...","[little, far, home]","[current, employee, less, 1, year]","[querétaro, querétaro]","[dec., 1, 2022]"
9998,"[use, great, company]",2.0,[amazon],[negative],[negative],[negative],"[become, excellent, problem, solver, use, data...","[cut-throat, management, toxic, culture, unnec...","[former, employee, 1, year]","[diego, ca]","[dec., 15, 2022]"


In [45]:
amzn['id'] = ids

In [46]:
amzn.to_csv('C:\\Users\\m.maraqa\\Desktop\\MMA courses\\Winter 2023\\Text Analytics\\Project\\Amazon_comments_processed.csv')